In [1]:
import funclib
import funclib.baselib as baselib
import funclib.iolib as iolib
import dblib.mssql as mssql
import csv

In [6]:
#GET FIELD TYPES OF CSV
#Prints field lengths as well
#Which can be used to create the database
#or to manually modify comments for my excel import

In [58]:
import pandas as pd

fname = 'C:/GISDATA/marine/gazetteer/medin/medin_gazetteer.csv'
t = {'medinid':pd.np.int32, 'name':pd.np.str, 'description':pd.np.str, 'feature_type':pd.np.str, 'x':pd.np.float16, 'y':pd.np.float16}
df = pd.read_csv(fname, delimiter=',', header=0, encoding = 'latin', dtype=t, na_filter=False, keep_default_na=False)
df.info(verbose=True)

for col in df:
    try:
        print('%s: %s' % (df[col].name, df[col].map(lambda x: len(x)).max()))
    except Exception as e:
        pass


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50163 entries, 0 to 50162
Data columns (total 7 columns):
medinid         50163 non-null int32
medinid_orig    50163 non-null int64
name            50163 non-null object
description     50163 non-null object
feature_type    50163 non-null object
x               50163 non-null float16
y               50163 non-null float16
dtypes: float16(2), int32(1), int64(1), object(3)
memory usage: 1.9+ MB
name: 59
description: 25
feature_type: 12


In [5]:
#MAIN IMPORT
#This takes a folder with 1 or more csv files in
#loops through them inserting records into an
#existing database

Reading rows for counter ....
update state


In [36]:
#This calculates how many rows we need to import
#Bearing in mind that it supports multiple csv sources
#in folder fld

#NOTE:
#For some imports, we can add the source file name
#and a candidate incremental key - adjust db and
#this import code accordingly

nr = 0
fld = 'C:/GISDATA/marine/gazetteer/medin'
dbname = 'gazetteer'
dbtable = 'medin'
server = '(local)'
#encoding = 'utf-8-sig'
encoding = 'latin'
has_header_row = True

fnr = sum(1 for fname in iolib.file_list_generator1(fld, '*.csv', recurse=True))
PP = iolib.PrintProgress(maximum=fnr, init_msg='Counting stuff...')

for fname in iolib.file_list_generator1(fld, '*.csv', recurse=True):
    with open(fname, encoding=encoding) as f:
        reader = csv.reader(f, delimiter=",")
        nr += sum(1 for row in reader)
    PP.increment()
    
print('Total nr:%s' %nr)

Counting stuff...
1 of 1 [##############################] 100.0% 

Total nr:50164


In [66]:
#see https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string
#for an explanation of the encoding
err_cnt = 0
PP = iolib.PrintProgress(maximum=nr, init_msg='Importing into SQL...')
errs = []
for fname in iolib.file_list_generator1(fld, '*.csv', recurse=True):
    _, nm, _ = iolib.get_file_parts2(fname)
    with mssql.Conn(dbname, server, autocommit=True) as cnn:
        with open(fname, encoding=encoding) as f:
            reader = csv.reader(f, delimiter=",")
            rw_nr = 0
            if has_header_row:
                is_header = True
            else:
                is_header = False
            for row in reader:
                rw_nr += 1
                if not is_header:
                    row = [s.replace("'","''") for s in row]
                    
                    #row.append(nm) #append filename
                    #row.append(str(rw_nr)) - used to create a key if we didnt have one
                    
                    ins = "'" + "','".join(row) + "'"
                    sql = "INSERT INTO [%s] SELECT %s" % (dbtable, ins)
                    
                    try:
                        with cnn.cursor() as cur:
                            cur.execute(sql)
                    except Exception as e:
                        err_cnt += 1
                        if err_cnt < 10:
                            print(e)
                        
                is_header = False
                PP.increment(suffix='Errs:%s' % err_cnt)
                    
print('Finished. Imported %s records of %s' % (nr - err_cnt, nr))

Importing into SQL...
50164 of 50164 [##############################] 100.0% Errs:0

Finished. Imported 50164 records of 50164
